# Control Systems 1, NB02: Introduction to System Classification and Properties
2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli


This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers the basics of system modeling, classification, and analysis, with a focus on building intuition through visual and interactive examples.

Authors:
- Mark Benazet Castells; mbenazet@ethz.ch
- Felix Muller; fmuller@ethz.ch

## Learning Objectives

After completing this notebook, you should be able to:

1. Understand input-output models of linear time-invariant (LTI) systems.
2. Write the model of an LTI system with A, B, C, D matrices.
3. Understand the concept of "state" to model a system.
4. Classify systems as static / dynamic, causal/non-causal, time-invariant/time-varying, linear/nonlinear.
5. Understand the notion of equilibrium points.
6. Approximate a non-linear system with a linear one about an equilibrium point.

# Setup


## Installing the required packages:


The following packages are required to run this notebook. If you haven't installed them yet, you can do so by running the following cell.

In [ ]:
%pip install numpy matplotlib scipy ipywidgets

## Import the packages:
The following cell imports the required packages. Run it before running the rest of the notebook.

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact, interactive, widgets, FloatSlider, Dropdown
from IPython.display import display, clear_output

# 1. Input-Output Models


In NB01 we saw that an input-output model is a map Σ from an input signal u(t) to an output signal y(t):

$$y = \Sigma u$$

or more explicitly:

$$y(t) = (\Sigma u)(t), \quad \forall t \in T$$

This can be represented by a block diagram:

<div style="text-align:center;">
    <img src="./img/block_dgm.png" alt="Block Diagram" width="500">
</div>

Depending on how Σ affects u, it is possible to classify the system in different ways:
- Linear vs. Nonlinear
- Causal vs. Non-causal
- Static (memoryless) vs. Dynamic
- Time-invariant vs. Time-varying

It is important to note, as we will see through the course, that certain tools are analysis techniques are required/useful only on certain types of systems. In general, in this course we focus on **Linear Time-invariant (LTI)** Systems. Thus, one should be able to determine whether a given system is already LTI or whether some transformation (e.g., non-linear → linear) is required.

# 2. Classification of Systems


## 1. Linear vs. Nonlinear Systems



A system is linear if it satisfies the superposition principle (i.e., is additive and homogenous):

$$\Sigma(a u_1 + b u_2) = a(\Sigma u_1) + b(\Sigma u_2)$$

Or in other words:

$$y(a u_1 + b u_2) = a y(u_1) + b y(u_2)$$

Where $\Sigma$ represents the system, $u = a u_1 + b u_2$ are control input signals, $y = a y_1 + b y_2$ are output signals, and a and b are scalars.

> As a core competency, given a system, you should be able to determine whether it is linear or non-linear. In general, the process is to determine whether the above superposition principle holds. 

The below examples, illustrate the general process:

### Example 1 - linear

Consider the system $y=3u$. To prove whether it is linear or non-linear:

**Step 1**: Let $u = a u_1 + b u_2$

**Step 2**: Substitute the above into $y$: $\quad y = 3 (a u_1 + b u_2)$

**Step 3**: Simplify the above to determine whether the superposition principle holds: 
\begin{align*}
y =& 3 a u_1 + 3 b u_2 + 4 \\
  =& a (3u_1) + b(3u_2) \\
  =& a (y_1) + b (y_2)
\end{align*}
Since the superposition principle holds, $y=3u$ is linear. 

### Example 2 - Non-linear

Consider the system $y = u^2$. To prove whether it is linear or non-linear:

**Step 1**: Let $u = a u_1 + b u_2$

**Step 2**: Substitute the above into $y$: $\quad y = (a u_1 + b u_2)^2$

**Step 3**: Simplify the above to determine whether the superposition principle holds: 
\begin{align*}
y &= (a u_1 + b u_2)^2 \\
    &= a^2 u_1^2 + 2 a b u_1 u_2 + b^2 u_2^2 \\
    &\neq a (u_1^2) + b (u_2^2)
\end{align*}

Since the superposition principle does not hold, $y = u^2$ is non-linear.

### Practical examples

The above examples may seem a bit abstract, so to help contextualize the origin of such equations (e.g., $y=ku, k\in\mathbb{R}_{\geq0}$), consider the below examples of practical systems:

A **simple electrical circuit** with a resistor is a linear system:
- The relationship between voltage (V) and current (I) follows Ohm's Law: $V = IR$.
- Combining two current sources, results in a voltage that is the sum of both current sources.

A **pendulum** in a grandfather clock is a non-linear system:
- Its motion is described by a non-linear equation: $\frac{d^2	\theta}{dt^2} + \frac{g}{L} \sin(	\theta) = 0$. 
- Doubling the initial displacement doesn't double the period or amplitude of oscillation.
- The behavior changes significantly with large displacements.
- Combining two initial conditions doesn't result in a simple combination of their individual motions.

Use the interactive plot below to check this property for different systems:

- Linear system: $y = u$
- Nonlinear system: $y = u^2$
- Nonlinear system: $y = \sin(u)$
- Nonlinear + Linear system: $y = \sin(u) + u$

Parameters:
- $a, b$: Scalars for the superposition principle

Adjust the parameters and observe:
- If the solid and dashed lines overlap for all inputs, the system is linear.
- If they diverge, the system is nonlinear.
- Is there a region of attraction around some values of $u$, where the non-linear and linear systems have similar outputs $y$?

In [ ]:
system_types = {
    'Linear: y = u': lambda u: u,
    'Nonlinear: y = u²': lambda u: u**2,
    'Nonlinear (Sine)': np.sin,
    'Nonlinear + Linear (Sine + Linear)': lambda u: np.sin(u) + u
}

def system_response(system_type, u):
    return system_types[system_type](u)

def check_linearity(system_type, a, b):
    u1 = np.linspace(-2, 2, 200)
    u2 = np.linspace(-2, 2, 200)
    
    # Individual responses
    y1 = system_response(system_type, u1)
    y2 = system_response(system_type, u2)
    
    # Linearity check
    y_sum = system_response(system_type, a*u1 + b*u2)
    y_individual_sum = a*y1 + b*y2
    
    fig, ax = plt.subplots()
    
    # Linearity plot with LaTeX labels
    ax.plot(u1, y_sum, label=r'$\Sigma(au_1 + bu_2)$')
    ax.plot(u1, y_individual_sum, label=r'$a\Sigma(u_1) + b\Sigma(u_2)$', linestyle='--')
    ax.set_title('Linearity Check')
    ax.set_xlabel('Input, $u$')
    ax.set_ylabel('Output, $y$')
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Create the interactive widgets
system_dropdown = Dropdown(
    options=list(system_types.keys()),
    value=list(system_types.keys())[0], 
    description='System'
)
a_slider = FloatSlider(min=-2, max=2, step=0.1, value=1)
b_slider = FloatSlider(min=-2, max=2, step=0.1, value=1)

# Create interactive plot
interactive_plot = interactive(check_linearity, system_type=system_dropdown, a=a_slider, b=b_slider)

# dipslay the widgets and plot
display(interactive_plot)

## 2. Causal vs Non-Causal Systems



1. **Causal System**: A system where the output at any time $t$ depends only on the values of the input on ($-\infty, t$]. 
   In other words, the system doesn't respond to future inputs.

2. **Non-Causal System**: A system where the output at time $t$ can depend on future inputs (after time $t$). 
   Such systems are not physically realizable in "the real world" as it would depend on future information.


A **thermostat** in your home is a causal system:
- It responds to current and past temperature readings.
- When the temperature drops below a set point, it turns on the heater.
- The thermostat's actions are based solely on present and past inputs, never on future temperatures.

While true non-causal systems don't exist in real-time physical world, we can find examples in data processing:

A **video editing software** applying a blur effect to an entire video clip is effectively non-causal:
- When blurring a frame at time t, it uses information from frames before and after t.
- This process "knows" about future frames, making it non-causal from the perspective of real-time video processing.

## 3. Static vs. Dynamic



1. **Static System**: A system where the output at time $t$ depends only on the input at time $t$.
   In other words, the system has no memory of past inputs.

2. **Dynamic System**: A system where the output at time $t$ depends on the input at time $t$ and past inputs.

A good example of a static system is a **volume knob on a speaker**. The volume level is always controlled by the current position of the knob, without any consideration of previous settings. No matter how much the knob was turned earlier, the current volume depends only on the knob's current position.

An example of a dynamic system is the **longitudinal speed of a car**. A specific depression of the gas pedal does not correspond to one specific speed, because the car's speed depends on both how much the gas pedal is currently pressed and how it has been pressed over time. Other factors such as inertia and friction also contribute, meaning the speed evolves dynamically based on the system's history.

## 4. Time-Invariant vs. Time-Varying



1. **Time-Invariant System**: A system where the input-output relationship does not change over time.
   In other words, the system's properties do not change with time.

2. **Time-Varying System**: A system where the input-output relationship changes over time.

Imagine a perfect actuator in a control system:

1. Ideal Motor: 
   This actuator consistently produces the same output for a given input, regardless of when the input is applied. It's like a brand-new, high-quality motor that responds identically today, tomorrow, or a year from now.

2. Aging Motor:
   In reality, actuators may degrade over time due to wear and tear. An aging motor might produce less output for the same input as time passes. This change in behavior over time characterizes a time-varying system.

<div style="text-align:center;">
    <img src="./img/actuator_decay.png" alt="Block Diagram" width="400">
</div>

> Given a system, you must be able to determine whether it is time-invariant or time-varying. 

### Example

To help understand the notation and general process, consider the simple system below:

$$ y(t) = t \cdot u(t) $$

Then, the two time shifted operations are:

$$
\begin{align*}
\left(\sigma_\tau \Sigma_u \right) \quad=\quad& y(t-\tau) &=\quad& (t-\tau) u(t-\tau) \\
\left(\Sigma \sigma_\tau u\right)(t) \quad=\quad& \left(\Sigma \tilde{u}\right)(t) &=\quad& t u(t-\tau)  
\end{align*}
$$

By noting that $(t-\tau) u(t-\tau) \neq t u(t-\tau)$, it is concludes that the system $y$ is time-varying. 

# 3. State-Space Models

In the previous notebook, NB01, the following general state-space representation of a system was introduced.  

$$
\begin{align*}
\dot{x}(t) &= f(t, x(t), u(t)) \\
y(t) &= g(t, x(t), u(t)) \\
\end{align*}
$$

Where:
- $x(t)$ is the state of the system.
- $u(t)$ is the control input.
- $y(t)$ is the output.

However, in this course, we will study only (finite-dimenional) Linear Time-Invariant (LTI) systems. These systems are linear and time-invariant, which means they satisfy the superposition principle and their properties do not change over time. Furthermore, although they are a restrictive class of system (i.e., most systems are not naturally LTI -- consider the problems from PS01), many systems can be approximated well by LTI models. Thus, the control methods taught in this course, albeit work mostly with LTI systems, are very powerful, and indeed more advanced and complex control system approaches are based on the principles covered in this course. 

Nevertheless, since we consider LTI systems, the equivalent (*linearized*) state-space model can be described by:
$$
\begin{align*}
\dot{x}(t) = Ax(t) + Bu(t) \\
y(t) = Cx(t) + Du(t) \\
\end{align*}
$$

Where:
- $A, B, C, D$ are matrices that define the system.

Although, most systems are not LTI, but many systems can be approximated very well by LTI models. On the [graph above](#1-linear-vs-nonlinear-systems), you can see that $\Sigma(a u_1 + b u_2)$ and $a\Sigma(u_1) + b\Sigma(u_2)$ are very similar in certain regions to the nonlinear systems, mainly around the origin. Thus, to transform a non-linear system to its linear equivalent, we **linearize** the system about these points to get the $A, B, C, D$ matrices.

These points, where a linearization is a faithful representation of the non-linear system, are called **equilibrium points**. For a general nonlinear state-space model, the equilibrium point $(x_e, u_e)$ is defined when:

$$f(x_e, u_e) = 0$$

> In general, given a system, you are expected to be able to compute a systems equilibrium points. Note that it is possible for a system to have multiple equilibrium points. Further, note that the linearization is only valid near the *equilibrium points*.

## 4. Linearization of a Nonlinear System



In general, a nonlinear system can be approximated by a linear model through a Taylor series expansion around its equilibrium point. The linearization process, in its most basic form, follows this structure:

### Given:
$$
\begin{cases}
\dot{x}(t) = f(x(t), u(t)) \\
y(t) = g(x(t), u(t))
\end{cases}
$$

### Wanted:
$$
\begin{cases}
\dot{x}(t) = A x(t) + B u(t) \\
y(t) = C x(t) + D u(t)
\end{cases}
$$

### Equilibrium point: 
Find the equilibrium points of the system.
$$(x_e, u_e)$$

### Linearization:
To compute the corresponding matrices $A, B, C, D$ about an equilibrium point, compute the respective jacobians, using the relevant equilibrium point, as defined below:

$$
\begin{align*}
A &= \begin{bmatrix}
\frac{\partial f_1}{\partial x_1} & \cdots & \frac{\partial f_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial f_n}{\partial x_1} & \cdots & \frac{\partial f_n}{\partial x_n}
\end{bmatrix}_{x_e, u_e}
\quad
&B& = \begin{bmatrix}
\frac{\partial f_1}{\partial u_1} \\
\vdots \\
\frac{\partial f_n}{\partial u_1}
\end{bmatrix}_{x_e, u_e} \\

C &= \begin{bmatrix}
\frac{\partial g_1}{\partial x_1} & \cdots & \frac{\partial g_1}{\partial x_n} 
\end{bmatrix}_{x_e, u_e}

\quad
&D& = \begin{bmatrix}
\frac{\partial g_1}{\partial u_1}
\end{bmatrix}_{x_e, u_e}
\end{align*}

$$

It is important to note that in the linearised state-space form in the resulting linear system, $x(t)$, $u(t)$ and $y(t)$ are not the original modeling coordinates, but rather represent the deviations from the equilibrium point.

To help concretise the above procedure, we provide a walkthrough example below. 

### Example:


We now explore the linearization of an example nonlinear system around an equilibrium point. This system is not a practical control example, but facilitates showing the effect of linearization visually. The system wish to analyze is the below:

$$
\begin{align*}
\dot x &=& f(x) &= \sin(x)  \\ 
y &=& g(x) &= x
\end{align*}
$$

First find the equilibrium points: $f(x)=0 \implies x_e = k\pi, k\in\mathbb{Z}_+$. 
For simplicity, let's pick the equilibrium point $x_e = \pi$ as the $x_e$ of interest.

Since there are no inputs, the two linearized matrices to compute are $A$ and $C$ by taking the corresponding derivatives:
$$
\begin{align*}
A &= \frac{d f(x)}{dx}\mid_{x=x_e} = \cos(x_e) = -1 \\
C &= \frac{d g(x)}{dx}\mid_{x=x_e} = 1 
\end{align*}
$$

Thus, the linear approximation of the system near $x_e = \pi$ is:

$$
\dot x = -1 \cdot (x-\pi) \\
y = 1 (x - \pi)
$$

#### Visualization

In the visualization, we plot both the linear ($f(x) = \sin(x_e)$) and non-linear ($f_l(x) = -1 (x_e - \pi)$) functions:
1. Plots the value $\dot x$, for a given $x_e$, comparing both the non-linear and linear representations -- note that the x-axis represents the delta to $x_e$  ($\delta x_e$). 

Note the following behaviors on the left plot:
- When $x_e = k\pi$, zoom in: Only the region near the linearization point is visible, and within this domain, the linear function is a faithful representation of the non-linear system.
- When $x_e = k\pi$, zoom out: Within the new domain, the linear system is no longer a faithful representation of the non-linear, even though we are still at the equilibrium point. 
- When $x_e \neq k\pi$: As you linearize about a point further away from $k\pi$, the region in which the linear system faithfully represents the non-linear one decreases. 
> This highlights that linearizing a system only returns faithful representations of non-linear systems when linearising about and operating near equilibrium points.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interactive, FloatSlider
from IPython.display import display

# Define the nonlinear system f(x) = sin(x)
def nonlinear_function(x):
    return np.sin(x)

# Calculate the linear approximation around the equilibrium point
def linear_approximation(x, x_e):
    slope = np.cos(x_e)
    return slope * (x - x_e) + np.sin(x_e)

# Plot the nonlinear system and its linearization
def plot_system(ax, zoom, x_e):
    # Invert zoom level for better visualization
    zoom = 1/zoom

    # Generate x values based on zoom level
    x_vals = np.linspace(x_e - zoom, x_e + zoom, 400)
    
    # Nonlinear system output
    y_nonlinear = nonlinear_function(x_vals)
    
    # Linear approximation at equilibrium point x0
    y_linear = linear_approximation(x_vals, x_e)
    
    # Plot the results on the provided axis
    ax.plot(x_vals, y_nonlinear, label=r'Nonlinear System: $f(x) = \sin(x)$', color='blue')
    ax.plot(x_vals, y_linear, label=rf'Linearised system: $f(x) = -1 (x-\pi)$', linestyle='--', color='red')
    
    # Highlight the equilibrium point
    ax.scatter(x_e, nonlinear_function(x_e), color='black', zorder=5)
    ax.text(x_e + 0.1, np.sin(x_e), f'Linearization Point ({x_e:.2f}, {np.sin(x_e):.2f})', fontsize=10)
    
    # Set plot details
    ax.set_title(rf'Nonlinear System when Linearized about $x_e$ = {x_e:.2f}')
    ax.set_xlabel(r'$\delta x_e$')
    ax.set_ylabel(r'f(x)')
    ax.legend()
    ax.grid(True)
    
    # Adjust x and y limits to keep linearization point in the middle
    ax.set_xlim(x_e - zoom, x_e + zoom)
    y_center = nonlinear_function(x_e)
    ax.set_ylim(y_center - zoom, y_center + zoom)

# Combined function to plot both subplots
def interactive_analysis(x_e, zoom):
    fig, ax = plt.subplots(figsize=(6, 4))  # Specify a size if needed

    # Plot the nonlinear system with linearization on the first subplot
    plot_system(ax, zoom, x_e)

    plt.tight_layout()
    plt.show()

# Create interactive widgets
interactive_plot = interactive(
    interactive_analysis, 
    x_e=FloatSlider(min=-np.pi, max=np.pi, step=0.01, value=0, description='xₑ'),
    zoom=FloatSlider(min=0.1, max=4.0, step=0.1, value=2.0, description='Zoom:')
)

display(interactive_plot)

# 4. Interactive Example

In this example we illustrate the effects of linearizing a non-linear system.

We consider a pendulum that is mounted to a wall and connected to a horizontal spring as shown below.
Let $l$ be the length of the pendulum, $J$ its moment of inertia, $m$ its mass, and let $\lambda$ represent the damping constant of the pendulum acting at the pivot (damped rotation due to friction).

Further, let $k$ represent the spring constant of the spring, and denote by $a$ the distance to the pivot of the point that connects the spring and the pendulum. Assume the spring to be initially relaxed at $\varphi =0$.
The system is actuated by an external force $F(t)$ which acts at a right angle to the pendulum.
There is a sensor measuring the angle $\varphi$ which we assume to be limited to $\varphi(t)\in\left(-\frac{\pi}{2},\frac{\pi}{2}\right)$.

<div style="text-align:center;">
    <img src="./img/pendel.png" alt="Block Diagram" width="400">
</div>

Note that for completeness below we include the system dynamics, the non-linear state-space representation, and the linearized representation. However, we also provide these as a practical pen-and-paper example in the Problem Sheet (Exercise 5). 

## System Dynamics



The differential equation governing the dynamics of the above described system is assumed to be given by

$$J  \cdot\ddot{\varphi}(t) = -mgl \sin \varphi(t) - \frac{a^2 k}{2} \sin 2\varphi(t) - \lambda \dot{\varphi}(t) + l F(t).$$

Further, let the system parameters be
$$
\begin{array}{rcl}
\begin{array}{rcl}
l &=& 1\text{ m}  & m &=& 1\text{ kg} \\[0.2em]
g &=& 10\text{ m/s}^2  & a &=& 0.5\text{ m} \\[0.2em]
k &=& 10\text{ N/m} & \lambda &=& 3\text{ Nms/rad} \\[0.2em]
J &=& 1\text{ Nms}^2/\text{rad} & &
\end{array}
\end{array}
$$

The initial conditions at $t=0$ are assumed to be such that pendulum is at angle $\varphi=0$ with angular velocity $\dot \varphi =0$.

## Nonlinear System Representation


Choosing the state vector $x(t) = \begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix} = \begin{bmatrix} \varphi(t) \\ \dot \varphi(t) \end{bmatrix}$, the input vector $u(t) = F(t)$, and the output vector $y(t) = \varphi(t)$, the system can be represented in state-space form as follows:

$$
\begin{array}{rcl}
\dot{x}(t) &=& f(x(t),u(t))\\
y(t) &=& g(x(t),u(t))
\end{array}
$$

For our specific case, this becomes:

$$
\begin{array}{rcl}
\dot{x}(t) &=& \begin{bmatrix} x_2(t) \\ \frac{-mgl \sin x_1(t) - \frac{a^2 k}{2} \sin 2x_1(t) - \lambda  x_2(t) + l u(t)}{J} \end{bmatrix}\\
y(t) &=& x_1(t)
\end{array}
$$

## Linearizing the System


The system is nonlinear, but we can linearize it around an equilibrium point. In this case, we will linearize it around the equilibrium point $\varphi = 0$, $\dot \varphi = 0$ and $F(t) = 0$.

The linearized system is given by:

$$
\begin{array}{rcl}
\dot{x}(t) &=& \begin{bmatrix} 0 & 1 \\ -\frac{l \cdot m \cdot g + a^2 \cdot k}{J} & -\frac{\lambda}{J} \end{bmatrix} x(t) + \begin{bmatrix} 0 \\ \frac{l}{J} \end{bmatrix} u(t) \\[0.5em]
y(t) &=& \begin{bmatrix} 1 & 0 \end{bmatrix} x(t)
\end{array}
$$




## Interactive Task


Below we simulate the pendulum system with the following control signal:
$$
F(t) = \begin{cases} 0  \qquad  0\leq t<10\\ F_1 \quad 10 \leq  t < 10 \\ F_2 \qquad 15 \leq t \end{cases}
$$

The control input $F(t)$ jumps from $0 N$ to $F_1 = 0.1 N$ at $10$ seconds, then to $F_2 = 1 N$ at $15$ seconds. 

Both the linearized and non-linear system are plotted over $20$ seconds. 
- Try increasing $F_1$ keeping $F_2 = 5N$, what do you notice about the two responses?
- Try increasing $F_2$, what do you notice about the two responses? What could be a cause of this? *Hint:* Consider the equilibrium points used in the linearization. 

We have some additional sliders that are attributes of the problem, feel free to play around with them and see how the output behaviours change.

In [ ]:
# System parameters
l = 1.0  # m
m = 1.0  # kg
g = 10.0  # m/s^2
a = 0.5  # m
k = 10.0  # N/m
lambda_ = 3.0  # Nms/rad
J = 1.0  # Nms^2/rad

# Control input F(t)
def F(t, F1, F2, t1, t2):
    if t < t1:
        return 0
    elif t < t2:
        return F1
    else:
        return F2

# Nonlinear system dynamics
def nonlinear_system(x, t, F1, F2, t1, t2):
    phi, phi_dot = x
    F_t = F(t, F1, F2, t1, t2)
    phi_ddot = (-l * m * g * np.sin(phi) - 0.5 * a**2 * k * np.sin(2*phi) - lambda_ * phi_dot + l * F_t) / J
    return [phi_dot, phi_ddot]

# Linear system dynamics
def linear_system(x, t, F1, F2, t1, t2):
    phi, phi_dot = x
    F_t = F(t, F1, F2, t1, t2)
    phi_ddot = (-l * m * g * phi - a**2 * k * phi - lambda_ * phi_dot + l * F_t) / J
    return [phi_dot, phi_ddot]

# Simulation and plotting function
def simulate_and_plot(F1, F2, t1, t2, phi0, phi_dot0):
    with output:
        clear_output(wait=True)  # Clear the previous output
        # Time array
        t = np.linspace(0, 20, 1000)

        # Initial conditions
        x0 = [phi0, phi_dot0]

        # Solve ODEs
        nonlinear_solution = odeint(nonlinear_system, x0, t, args=(F1, F2, t1, t2))
        linear_solution = odeint(linear_system, x0, t, args=(F1, F2, t1, t2))

        # Extract phi and phi_dot
        nonlinear_phi = nonlinear_solution[:, 0]
        linear_phi = linear_solution[:, 0]

        # Calculate F(t) for plotting
        F_t = np.array([F(ti, F1, F2, t1, t2) for ti in t])

        # Plotting
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle('Pendulum System Simulation: Nonlinear vs Linear', fontsize=16, y=1.05)

        # Plot phi
        ax1.plot(t, nonlinear_phi, label='Nonlinear')
        ax1.plot(t, linear_phi, label='Linear', linestyle='--')
        ax1.set_xlabel(r'Time (s)')
        ax1.set_ylabel(r'Angle (rad)')
        ax1.set_title(r'Pendulum Angle ($\varphi$)')
        ax1.grid(True)
        ax1.legend()

        # Plot F(t)
        ax2.plot(t, F_t)
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('Force (N)')
        ax2.set_title(r'Control Input $F(t)$')
        ax2.grid(True)

        plt.tight_layout()
        plt.subplots_adjust(top=0.85, wspace=0.3)
        plt.show()

# Initialize a display object
output = widgets.Output()


# Create widgets
F1_slider = widgets.FloatSlider(value=0.1, min=0, max=1, step=0.1, description='F₁:')
F2_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='F₂:')
t1_slider = widgets.FloatSlider(value=10, min=0, max=20, step=0.1, description='t₁:')
t2_slider = widgets.FloatSlider(value=15, min=0, max=20, step=0.1, description='t₂:')
phi0_slider = widgets.FloatSlider(value=0.0, min=-np.pi/2, max=np.pi/2, step=0.01, description='ϕ₀:')
phi_dot0_slider = widgets.FloatSlider(value=0.0, min=-5, max=5, step=0.01, description='dϕ₀/dt:')

# Arrange sliders in columns
column1 = widgets.VBox([F1_slider, F2_slider])
column2 = widgets.VBox([t1_slider, t2_slider])
column3 = widgets.VBox([phi0_slider, phi_dot0_slider])

# Combine columns into a row
slider_layout = widgets.HBox([column1, column2, column3])

# Create interactive output
widgets.interactive_output(simulate_and_plot, {
    'F1': F1_slider,
    'F2': F2_slider,
    't1': t1_slider,
    't2': t2_slider,
    'phi0': phi0_slider,
    'phi_dot0': phi_dot0_slider
})

# Display the sliders and interactive plot
display(slider_layout, output)